In [ ]:
import pandas as pd

melbourne_file_path = '../data/melb_data.csv'

melbourne_data = pd.read_csv(melbourne_file_path)
print(melbourne_data.shape)
print(melbourne_data.columns)

In [ ]:
melbourne_data = melbourne_data.dropna(axis=0)
print(melbourne_data.shape)
print(melbourne_data.columns)
y = melbourne_data.Price
print("y", y)

melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X = melbourne_data[melbourne_features]
print('x description', X.describe())
print('x head', X.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
melbourne_model = DecisionTreeRegressor(random_state=1)

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

# Fit model
melbourne_model.fit(train_X, train_y)

In [ ]:
from sklearn.metrics import mean_absolute_error


print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
predicted_home_prices = melbourne_model.predict(val_X)
print("top value: ", val_y[0:5], predicted_home_prices[0:5])
mean_absolute_error(val_y, predicted_home_prices)